<a href="https://colab.research.google.com/github/guillermocalderon2021/03MIAR_Algoritmos_de_Optimizacion/blob/main/Algoritmos_GuillermoCalderon_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Actividad guiada 3**
## **Nombre:** Guillermo de Jesús Calderón
**Repositorio:** https://github.com/guillermocalderon2021/03MIAR_Algoritmos_de_Optimizacion/blob/main/Algoritmos_GuillermoCalderon_AG3.ipynb

**Nota:** Se realiza ejercicio para optar a puntuación extra

#Carga de librerias

In [6]:
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.8 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1


#Carga de los datos del problema

In [7]:
import urllib.request #Hacer llamadas http a paginas de la red
import tsplib95       #Modulo para las instancias del problema del TSP
import math           #Modulo de funciones matematicas. Se usa para exp
import random                     #Para generar valores aleatorios
import copy

#Descargamos el fichero de datos(Matriz de distancias)
file = "swiss42.tsp" ;
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprimir el fichero de datos



In [8]:
#Carga de datos y generación de objeto problem
###############################################################################
problem = tsplib95.load(file)

#Nodos
Nodos = list(problem.get_nodes())

#Aristas
Aristas = list(problem.get_edges())



In [95]:
#Probamos algunas funciones del objeto problem

#Distancia entre nodos
print(problem.get_weight(0, 1))

#Todas las funciones
#Documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html

#dir(problem)
#list(problem.get_edges())

15


#Funciones basicas


In [96]:

#Funciones basicas
###############################################################################

#Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set(solucion)))]
  return solucion

#Devuelve la distancia entre dos nodos
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#Devuelve la distancia total de una trayectoria/solucion
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)




#BUSQUEDA ALEATORIA

In [97]:
###############################################################################
# BUSQUEDA ALEATORIA
###############################################################################

def busqueda_aleatoria(problem, N):
  #N es el numero de iteraciones
  Nodos = list(problem.get_nodes())

  mejor_solucion = []
  #mejor_distancia = 10e100                         #Inicializamos con un valor alto
  mejor_distancia = float('inf')                    #Inicializamos con un valor alto

  for i in range(N):                                #Criterio de parada: repetir N veces pero podemos incluir otros
    solucion = crear_solucion(Nodos)                #Genera una solucion aleatoria
    distancia = distancia_total(solucion, problem)  #Calcula el valor objetivo(distancia total)

    if distancia < mejor_distancia:                 #Compara con la mejor obtenida hasta ahora
      mejor_solucion = solucion
      mejor_distancia = distancia


  print("Mejor solución:" , mejor_solucion)
  print("Distancia     :" , mejor_distancia)
  return mejor_solucion


#Busqueda aleatoria con 20000 iteraciones
solucion = busqueda_aleatoria(problem, 20000)

Mejor solución: [0, 1, 16, 31, 34, 9, 39, 36, 35, 15, 26, 23, 40, 18, 3, 8, 38, 33, 28, 21, 24, 10, 25, 11, 6, 17, 37, 19, 41, 14, 7, 27, 2, 32, 4, 22, 29, 13, 12, 5, 20, 30]
Distancia     : 3621


#BUSQUEDA LOCAL

In [98]:
###############################################################################
# BUSQUEDA LOCAL
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos) Si hay N nodos se generan (N-1)x(N-2)/2 soluciones
  #Se puede modificar para aplicar otros generadores distintos que 2-opt
  #print(solucion)
  mejor_solucion = []
  mejor_distancia = 10e100
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  return mejor_solucion


#solucion = [1, 47, 13, 41, 40, 19, 42, 44, 37, 5, 22, 28, 3, 2, 29, 21, 50, 34, 30, 9, 16, 11, 38, 49, 10, 39, 33, 45, 15, 24, 43, 26, 31, 36, 35, 20, 8, 7, 23, 48, 27, 12, 17, 4, 18, 25, 14, 6, 51, 46, 32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))


nueva_solucion = genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))


Distancia Solucion Incial: 3621
Distancia Mejor Solucion Local: 3323


In [101]:
#Busqueda Local:
#  - Sobre el operador de vecindad 2-opt(funcion genera_vecina)
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local(problem):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      #mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      print(len(mejor_solucion))
      return mejor_solucion

    solucion_referencia = vecina


sol = busqueda_local(problem )

En la iteracion  38 , la mejor solución encontrada es: [0, 8, 23, 9, 39, 22, 38, 33, 20, 34, 30, 29, 28, 2, 27, 3, 4, 10, 25, 41, 40, 24, 21, 14, 37, 17, 35, 36, 15, 16, 19, 13, 5, 26, 11, 12, 18, 6, 7, 31, 32, 1]
Distancia     : 1893
42


# **Mejoras al algoritmo**
Para mejorar el algoritmo desarrollado en clase se usó la **metaheuristica GRASP**. A continuación se explican las modificaciones realizadas al algoritmo:

1. Para generar las soluciones iniciales mediante una estrategia "voraz" creamos creamos una lista restringida de candidatos mediante los elementos que cumplan que $x\le (1-\alpha) min+\alpha  max$ con $x\in [0,1]$.

2. Para generar el **multiarranque** creamos 19 soluciones variando el parametro $\alpha$ desde 0.05 hasta 0.95 con saltos de 0.05. Esto nos permite diversificar sobre zonas del espacio de búsqueda poco o nada visitadas.

3. Para intensificar se está realizando una Busqueda de Entorno Variable (VNS) usando consecutivamente tres operadores: a) Intercambio, b) Inserción, c) Reordenamiento (Barajeo).

In [102]:

def generar_solucion_grasp(alpha):
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(crear_rcl(alpha,solucion))]
  return solucion

def crear_rcl(alpha,sol_parcial):
  elegibles=list(set(Nodos) - set(sol_parcial))
  max_weight=max(list(map(lambda x: problem.get_weight(sol_parcial[-1],x), elegibles)))
  min_weight=min(list(map(lambda x: problem.get_weight(sol_parcial[-1],x), elegibles)))
  rcl=list(filter(lambda x:  problem.get_weight(sol_parcial[-1],x) <= (1-alpha)*min_weight+alpha*max_weight, elegibles))
  return rcl

sol=generar_solucion_grasp(0.05)
print(sol)
print(distancia_total(sol,problem))

[0, 1, 4, 2, 27, 28, 30, 29, 8, 9, 23, 41, 10, 25, 11, 12, 18, 26, 5, 6, 3, 7, 14, 16, 15, 37, 17, 31, 20, 33, 34, 32, 38, 22, 39, 21, 40, 24, 13, 19, 36, 35]
1671


In [103]:
###############################################################################
# BUSQUEDA EN ENTORNOS VARIABLES
###############################################################################
def genera_vecina(solucion):
  #Generador de soluciones vecinas usando 3 operadores: a) Intercambio, b) Insercion, c) Barajeo
  mejor_solucion = []
  mejor_distancia = 10e100
  # SWAP
  for i in range(1,len(solucion)-1):          #Recorremos todos los nodos en bucle doble para evaluar todos los intercambios 2-opt
    for j in range(i+1, len(solucion)):

      #Se genera una nueva solución intercambiando los dos nodos i,j:
      #  (usamos el operador + que para listas en python las concatena) : ej.: [1,2] + [3] = [1,2,3]
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)

      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  #print(f'{mejor_solucion}-{mejor_distancia} SWAP')
  #INSERTIONS
  vecina=copy.copy(mejor_solucion)
  for i in range(0,len(solucion)):
    item=vecina.pop(i)
    for j in range(i+1, len(solucion)):
      vecina.insert(j,item)
       #Se evalua la nueva solución ...
      distancia_vecina = distancia_total(vecina, problem)
      #print(f'{vecina}-{distancia_vecina}-{mejor_distancia}')
      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = copy.copy(vecina)
      vecina.pop(j)
    vecina.insert(i,item)
  #print(f'{mejor_solucion}-{mejor_distancia}-INSERT')
  #BARAJEO
  solucion=copy.copy(mejor_solucion)
  for i in range(1,len(solucion)):
    for j in range(i+1,len(solucion)-1,i+1):
      copia=solucion[i:j+1]
      vecina=solucion[:i]+ random.sample(copia,len(copia))+solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)
      #... para guardarla si mejora las anteriores
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina
        mejor_solucion = vecina
  #print(f'{mejor_solucion}-{mejor_distancia} REORDER')
  return mejor_solucion


In [104]:
#Busqueda en entornos variables:
#  - Usando 3 operadores de vecindad : 1)Intercambio, 2) Insercion, 3) Barajeo
#  - Sin criterio de parada, se para cuando no es posible mejorar.
def busqueda_local_grasp(problem,alpha):
  mejor_solucion = []

  #Generar una solucion inicial de referencia(aleatoria)
  solucion_referencia = crear_solucion(Nodos)
  mejor_distancia = distancia_total(solucion_referencia, problem)

  iteracion=0             #Un contador para saber las iteraciones que hacemos
  while(1):
    iteracion +=1         #Incrementamos el contador
    #print('#',iteracion)

    #Obtenemos la mejor vecina ...
    vecina = genera_vecina(solucion_referencia)

    #... y la evaluamos para ver si mejoramos respecto a lo encontrado hasta el momento
    distancia_vecina = distancia_total(vecina, problem)

    #Si no mejoramos hay que terminar. Hemos llegado a un minimo local(según nuestro operador de vencindad 2-opt)
    if distancia_vecina < mejor_distancia:
      mejor_solucion = copy.deepcopy(vecina)   #Con copia profunda. Las copias en python son por referencia
      #mejor_solucion = vecina                   #Guarda la mejor solución encontrada
      mejor_distancia = distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia     :" , mejor_distancia)
      return mejor_solucion

    solucion_referencia = vecina

In [106]:
import numpy as np

def solucion_tsp_grasp():
  mejor_solucion_global=[]
  mejor_distancia_global=float('inf')
  mejor_alpha=0
  for alfa in np.arange(0.05,1,0.05):
    print(f'---------------------- Con alpha = {alfa} --------------------')
    sol = busqueda_local_grasp(problem,alfa )
    distancia=distancia_total(sol,problem)
    if distancia < mejor_distancia_global:
      mejor_solucion_global = copy.copy(sol)
      mejor_distancia_global = distancia
      mejor_alpha=alfa
  print('__________________________________________________________________________________________________________')
  print(f'MEJOR SOLUCION GLOBAL: {mejor_solucion_global}')
  print(f'Encontrada usando alpha = {mejor_alpha} ')
  print(f'Distancia: {mejor_distancia_global}')
  print('__________________________________________________________________________________________________________')

solucion_tsp_grasp()

---------------------- Con alpha = 0.05 --------------------
En la iteracion  18 , la mejor solución encontrada es: [10, 41, 23, 40, 24, 29, 28, 4, 13, 19, 14, 16, 15, 37, 7, 17, 36, 35, 31, 32, 30, 38, 22, 39, 21, 9, 8, 25, 11, 12, 18, 26, 5, 6, 1, 0, 20, 33, 34, 27, 2, 3]
Distancia     : 1655
---------------------- Con alpha = 0.1 --------------------
En la iteracion  18 , la mejor solución encontrada es: [0, 7, 37, 36, 35, 31, 17, 30, 29, 21, 24, 40, 23, 41, 25, 11, 12, 18, 10, 8, 9, 39, 22, 38, 33, 20, 34, 32, 28, 27, 2, 4, 6, 26, 5, 13, 19, 14, 16, 15, 1, 3]
Distancia     : 1583
---------------------- Con alpha = 0.15000000000000002 --------------------
En la iteracion  19 , la mejor solución encontrada es: [28, 30, 38, 22, 39, 21, 24, 40, 8, 14, 16, 15, 37, 20, 33, 34, 32, 1, 6, 5, 13, 19, 7, 31, 35, 36, 17, 0, 27, 29, 9, 23, 41, 10, 25, 11, 12, 18, 26, 4, 3, 2]
Distancia     : 1675
---------------------- Con alpha = 0.2 --------------------
En la iteracion  23 , la mejor solució